In [1]:
import sys
import os
%load_ext autoreload
%autoreload 2

sys.path.append('../')
os.chdir('../')

In [2]:
import re

string = "res=[0, 1, 2, 5]"
pattern = r"res=\[(.*?)\]"
result = re.findall(pattern, string)[0]
lst = [int(num) for num in result.split(", ")]
print(lst)


[0, 1, 2, 5]


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re


global_path = './outputs'
all_result_paths = glob.glob(global_path + '/*_exp/**/result.csv', recursive=True)
result_summary = {}
n_layers = 6
for result_path in all_result_paths:
    result = pd.read_csv(result_path)
    top_test_result = result['test_acc'].max()
    pe_type = 'NoPE' if 'nope' in result_path else 'Original'
    pattern = r"res=\[(.*?)\]"
    residual_list = re.findall(pattern, result_path)[0]
    residual_list = [int(num) for num in residual_list.split(", ")]
    missing_rc = [i for i in range(n_layers) if i not in residual_list]
    missing_rc_str = '{' + ','.join([str(i) for i in missing_rc])+ '}'


    if pe_type not in result_summary:
        result_summary[pe_type] = {}
    if missing_rc_str not in result_summary[pe_type]:
        result_summary[pe_type][missing_rc_str] = []
    result_summary[pe_type][missing_rc_str].append(top_test_result)
    if pe_type == 'Original' and missing_rc_str == '{5}':
        print(result_path, pe_type, missing_rc_str, top_test_result)


./outputs/add3_residual_exp/add3_sd240_T2403191320_res=[0, 1, 2, 3, 4]/add3_sd240_T2403191320_res=[0, 1, 2, 3, 4]/result.csv Original {5} 99.61616161616162
./outputs/add3_residual_exp/add3_sd241_T2403191954_res=[0, 1, 2, 3, 4]/add3_sd241_T2403191954_res=[0, 1, 2, 3, 4]/result.csv Original {5} 99.62626262626264
./outputs/add3_residual_exp/add3_sd242_T2403200235_res=[0, 1, 2, 3, 4]/add3_sd242_T2403200235_res=[0, 1, 2, 3, 4]/result.csv Original {5} 97.11111111111111
./outputs/add3_residual_exp/add3_sd243_T2403200838_res=[0, 1, 2, 3, 4]/add3_sd243_T2403200838_res=[0, 1, 2, 3, 4]/result.csv Original {5} 99.04040404040404
./outputs/add3_residual_exp/add3_sd244_T2403201641_res=[0, 1, 2, 3, 4]/add3_sd244_T2403201641_res=[0, 1, 2, 3, 4]/result.csv Original {5} 99.75757575757576


In [ ]:
result_summary

In [9]:
from ast import literal_eval
mean_results = {}
for pe_type in result_summary:
    cur_type_dict = result_summary[pe_type].copy()
    for missing_rc in cur_type_dict:
        cur_type_dict[missing_rc] = len(cur_type_dict[missing_rc])
    mean_results[pe_type] = cur_type_dict
df = pd.DataFrame(mean_results)
missing0 = ['{}']
missing1 = [str({i}) for i in range(n_layers)]
missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
from IPython.display import display
display(df.loc[missing0,:].T)
display(df.loc[missing1,:].T)
display(df.loc[missing2,:].T)
display(df.loc[missing3,:].T)

,{}
Original,5
NoPE,5


,{0},{1},{2},{3},{4},{5}
Original,5,5,5,5,5,5
NoPE,5,5,5,5,5,5


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
Original,5,5,5,5,5
NoPE,5,5,4,5,5


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
Original,5,5,5,5
NoPE,5,5,5,5


In [11]:
from ast import literal_eval
from collections import OrderedDict
final_results = OrderedDict()
for pe_type in {'Original', 'NoPE'}:
    cur_type_dict = result_summary[pe_type].copy()
    final_dict = {
        'min': cur_type_dict.copy(),
        'max': cur_type_dict.copy(),
        'avg.': cur_type_dict.copy(),
    }
    for missing_rc in cur_type_dict:
        final_dict['avg.'][missing_rc] = np.mean(cur_type_dict[missing_rc])
        final_dict['min'][missing_rc] = np.min(cur_type_dict[missing_rc])
        final_dict['max'][missing_rc] = np.max(cur_type_dict[missing_rc])

    for k in final_dict:
        final_results[pe_type+' ('+k+')'] = final_dict[k]
df = pd.DataFrame(final_results)
missing1 = [str({i}) for i in range(n_layers)]

missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
from IPython.display import display
r1 = df.loc[missing1,:].T
r2 = df.loc[missing2,:].T
r3 = df.loc[missing3,:].T
r0 = df.loc[['{}'],:].T
display(r0)
display(r1)
display(r2)
display(r3)

# write all 3 to the same csv file
all_r = pd.concat([r0, r1, r2, r3], axis=1)
all_r.to_csv('all_r_mean.csv')


,{}
NoPE (min),86.595960
NoPE (max),94.656566
NoPE (avg.),91.600000
Original (min),99.838384
Original (max),99.959596
Original (avg.),99.907071


,{0},{1},{2},{3},{4},{5}
NoPE (min),87.414141,38.939394,34.010101,1.454545,21.959596,92.767677
NoPE (max),92.141414,88.646465,56.868687,32.525253,85.242424,96.969697
NoPE (avg.),90.191919,70.674747,47.680808,13.967677,63.646465,95.311111
Original (min),99.808081,98.404040,99.292929,97.858586,94.919192,97.111111
Original (max),99.979798,99.707071,99.848485,99.959596,99.727273,99.757576
Original (avg.),99.931313,99.193939,99.565657,99.476768,98.472727,99.030303


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
NoPE (min),0.737374,0.010101,0.050505,0.020202,0.020202
NoPE (max),7.333333,0.060606,0.070707,0.101010,0.121212
NoPE (avg.),2.963636,0.028283,0.060606,0.052525,0.040404
Original (min),83.818182,0.010101,0.010101,0.020202,0.020202
Original (max),99.131313,98.898990,0.050505,0.080808,0.030303
Original (avg.),93.553535,19.820202,0.024242,0.036364,0.024242


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
NoPE (min),0.010101,0.010101,0.010101,0.020202
NoPE (max),0.030303,0.050505,0.050505,0.050505
NoPE (avg.),0.020202,0.026263,0.024242,0.026263
Original (min),0.010101,0.010101,0.010101,0.020202
Original (max),0.050505,0.040404,0.050505,0.040404
Original (avg.),0.026263,0.022222,0.026263,0.026263
